In [2]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import textwrap 
import os.path
import urllib.request

In [ ]:
%%appyter markdown

# KINOMEScan Data Visualization

This appyter creates bar charts for the visualization of KINOMEScan data. If a small molecule is inputted, it will return 
a list of kinases that small molecule has been found to bind to from KINOMEScan assays. 
It will also generate an interactive bar chart displaying the kinases sorted by their equilibrium dissociation constant (an indication of binding affinity.)
Similarly, if a kinase is inputted, it will return a list of the small molecules that have been found to bind to it. 
It will also generate an interactive bar chart displaying the small molecules sorted by their equilibrium dissociation constant. 

In [24]:
%%appyter hide_code

{% do SectionField(name='title', title = 'Visualize KINOMEScan Data')%}

{% do SectionField(name = 'section', title = 'Input a Small Molecule and/or Kinase', 
                   subtitle = 'Input a small molecule to visualize the kinases that it binds to, ' + 
                   'and/or input a kinase to visualize the small molecules that bind it.')%}   

{% do SectionField(name = 'section2', title = 'Upload or Enter a Gene/Protein List', 
                   subtitle = 'Upload or enter a gene/protein list (one gene per row) to visualize the ' +
                   'small molecules that bind to the protein kinases in that list.')%}  

In [ ]:
%%appyter markdown

### Generate list of kinases for small molecule input

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity

sm_name = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name',
        description = 'The small molecule name',
        default = 'Afatinib',
        section = 'section'
    )}}

kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID
hms_id = kinome_df.loc[kinome_df['sm_name'] == sm_name, 'sm_hms_id']

# remove the row number
hms_id_list = []
for hms_id in hms_id:
    hms_id_list.append(hms_id)
    
hms_id_string = str(hms_id_list[0])
old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'
new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 
data = pd.read_csv(new_url)
df = pd.DataFrame(data)

target_affinity_1 = df.loc[df['Binding Class'] == 1, 'HUGO Gene Symbol']
target_affinity_2 = df.loc[df['Binding Class'] == 2, 'HUGO Gene Symbol']
target_affinity_3 = df.loc[df['Binding Class'] == 3, 'HUGO Gene Symbol']
target_affinity_10 = df.loc[df['Binding Class'] == 10, 'HUGO Gene Symbol'] 

#js = Javascript('alert("Invalid input")')
kinase1_list = []
kinase2_list = []
kinase3_list = []

for kinase in target_affinity_1:
    if kinase not in kinase1_list: # removes duplicates
        kinase1_list.append(str(kinase))

if len(kinase1_list) != 0:
    print(sm_name + ' binds to the following kinases with Kd < 100 nM: ')
    kinase1_string = ', '.join(kinase1_list)
    print(kinase1_string)
    print()

for kinase in target_affinity_2:
    if kinase not in kinase2_list: # removes duplicates
        kinase2_list.append(str(kinase))

if len(kinase2_list) != 0:
    print(sm_name + ' binds to the following kinases with 100 nM ≤ Kd < 1µM: ')
    kinase2_string = ', '.join(kinase2_list)
    print(kinase2_string)
    print()

for kinase in target_affinity_3:
    if kinase not in kinase3_list: # removes duplicates
        kinase3_list.append(str(kinase))

if len(kinase3_list) != 0:
    print(sm_name + " binds to the following kinases with 1µM ≤ Kd < 10 µM: ")
    kinase3_string = ', '.join(kinase3_list)
    print(kinase3_string)

In [ ]:
%%appyter markdown

### Generate bar chart for small molecule input
Hover over bar(s) to see kinases

In [ ]:
x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
y = [len(kinase1_list), len(kinase2_list), len(kinase3_list)]

# Text wraparound
kinase1_lab = '<br>'.join(textwrap.wrap(kinase1_string, width=50))
kinase2_lab ='<br>'.join(textwrap.wrap(kinase2_string, width=50))
kinase3_lab ='<br>'.join(textwrap.wrap(kinase3_string, width=50))

fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                             hovertext = [kinase1_lab, kinase2_lab, kinase3_lab], 
                             hoverlabel = dict(font = dict(size = 18)),
                             marker = {'color': y,
                                        'colorscale': 'Plotly3'})])

fig.update_layout(
    title = {
        'text': 'Kinases bound by ' + sm_name,
        'y':0.87,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title = "Equilibrium Dissociation Constant",
    yaxis_title = "Kinases",
    font = dict(
        #family = "Courier New, monospace",
        size = 18,
        color = 'black'
    )
)

fig.show()

In [ ]:
%%appyter markdown

### Generate list of small molecules for kinase input
Includes download of the KINOMEscan GMT files. For more information on how this was assembled, go to 
the KinomeScan-Appyter folder in the HarminozomePythonScripts repository
(https://github.com/MaayanLab/HarmonizomePythonScripts). 

In [ ]:
%%appyter code_exec

# if kinase inputted, return small molecules  
kinase_name = {{ StringField(
        name = 'Kinase name',
        label = 'Kinase name',
        description = 'The kinase name',
        default = 'ABL1',
        section = 'section'
    )}}



if not os.path.exists('level1.gmt'):
    urllib.request.urlretrieve('https://gist.github.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb#file-level1-gmt', 'level1.gmt')
level_1 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level1.gmt', 'r'))}
if not os.path.exists('level2.gmt'):
    urllib.request.urlretrieve('https://gist.github.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb#file-level2-gmt', 'level2.gmt')
level_2 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level2.gmt', 'r'))}
if not os.path.exists('level3.gmt'):
    urllib.request.urlretrieve('https://gist.github.com/serena-zhang/a5344564a9beed30e7b5a626da1c0deb#file-level3-gmt', 'level3.gmt')
level_3 = {split_line[0]: split_line[2:] for split_line in map(lambda s: s.split('\t'), open('level3.gmt', 'r'))}

# lists for each target affinity for the small molecules
sm1_list = []
sm2_list = []
sm3_list = []

if kinase_name in level_1:
    sm1_list = level_1[kinase_name]
if kinase_name in level_2:
    sm2_list = level_2[kinase_name]
if kinase_name in level_3:
    sm3_list = level_3[kinase_name]
    
if len(sm1_list) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with Kd < 100 nM: ')
    sm1_string = ', '.join(sm1_list)
    print(sm1_string)
    print()
    
if len(sm2_list) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 100 nM ≤ Kd < 1µM: ')
    sm2_string = ', '.join(sm2_list)
    print(sm2_string)    
    print()
    
if len(sm3_list) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 1µM ≤ Kd < 10 µM: ')
    sm3_string = ', '.join(sm3_list)
    print(sm3_string)    
    print()

In [ ]:
# Bar graph for the small molecules 
x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
y = [len(sm1_list), len(sm2_list), len(sm3_list)]

# Text wraparound
sm1_lab = '<br>'.join(textwrap.wrap(sm1_string, width=50))
sm2_lab ='<br>'.join(textwrap.wrap(sm2_string, width=50))
sm3_lab ='<br>'.join(textwrap.wrap(sm3_string, width=50))

fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                             hovertext = [sm1_lab, sm2_lab, sm3_lab], 
                             hoverlabel = dict(font = dict(size = 18)),
                             marker = {'color': y,
                                        'colorscale': 'Plotly3'})])

fig.update_layout(
    title = {
        'text': 'Small molecules that bind ' + kinase_name,
        'y':0.87,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title = "Equilibrium Dissociation Constant",
    yaxis_title = "Number of small molecules",
    font = dict(
        #family = "Courier New, monospace",
        size = 18,
        color = 'black'
    )
)

fig.show()

In [ ]:
%%appyter code_exec

# Import gene list as file or from text box file
# Will choose file upload over textbox if a file is given 

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        description = 'Gene/protein list upload',
        default = '',
        section = 'section2'
    )}}

gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Gene/protein list',
        description = 'Gene/protein list input',
        default = 'AAK1 \nABL1 \nABL2 \nACVR1',
        section = 'section2'
    )}}

if gene_list_file != '':
    open_gene_list_file = open(gene_list_filename,'r')
    lines = open_gene_list_file.readlines()
    genes = [x.strip() for x in lines]
    open_gene_list_file.close()
else:
    genes = gene_list_input.split('\n')
    genes = [x.strip() for x in genes]
    
# Only retrieve the protein kinases from the protein/gene list
prot_kinases = []
for gene in genes:
    if ((gene in level_1) or (gene in level_2) or (gene in level_3)): # if gene is a kinase 
        prot_kinases.append(gene)

# go through prot_kinases and retrieve the small molecules that were studied, then make a list of all the sm
sm_all = []
for kinase in prot_kinases:
    